In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import unicodedata
import time
from tqdm import tqdm
import re
import sys
import requests
import warnings
import pandas as pd

from time import sleep
from copy import deepcopy
from tqdm import tqdm_notebook as tqdm
from IPython.display import Markdown, display

In [9]:
# Print Timestamp At time of crawl
datePosted = str(datetime.date.today())
print('Time of Crawl: ' + datePosted)

Time of Crawl: 2020-07-29


In [10]:
warnings.filterwarnings('ignore')

def printmd(string):
    display(Markdown(string))

printmd("<span style='font-family:courier;color:red;font-size:115%;' >**--> Libraries Imported Successfully.**</span>")

<span style='font-family:courier;color:red;font-size:115%;' >**--> Libraries Imported Successfully.**</span>

In [11]:
print("\n===========================")
print("Fetching University Page(s) :")
print("===========================")

base_url = 'https://pureportal.coventry.ac.uk/en/persons/?format='

total_pages = 10

uni_urls = []

for x in tqdm(range(total_pages)):
    uni_urls.append(base_url+str(x))
    
no_of_profiles = total_pages*50
    
printmd("<span style='font-family:courier;color:red;font-size:115%;' > **--> There are Total "+str(total_pages)+" university page(s) found, each page(s) contains 50 researchers profile links.**</span>")


Fetching University Page(s) :


<span style='font-family:courier;color:red;font-size:115%;' > **--> There are Total 10 university page(s) found, each page(s) contains 50 researchers profile links.**</span>

In [12]:
print("\n================================")
print("Fetching links of All Profiles : ")
print("================================")

prof_links = []

for uni_url in tqdm(uni_urls):
    request = requests.get(uni_url)
    soup = BeautifulSoup(request.content, 'html.parser')    
    result_grid = soup.find('section', { "class" : "grid-results"})
    
    for a in result_grid.find_all('a', { "class" : "link person" }, href=True):
        prof_links.append(a['href'])


printmd("<span style='font-family:courier;color:red;font-size:115%;' > **--> Total "+str(len(prof_links))+"  Numbers of Researchers Profile Found.**</span>")


Fetching links of All Profiles : 


<span style='font-family:courier;color:red;font-size:115%;' > **--> Total 500  Numbers of Researchers Profile Found.**</span>

In [13]:
print("\n=====================================")
print("Fetching Details of Each Researcher : ")
print("=====================================")

#prof_link = ['https://research-information.bris.ac.uk/en/persons/david-w-f-abbott', 'https://research-information.bris.ac.uk/en/persons/ana-paula-abdala-sheikh', 'https://research-information.bris.ac.uk/en/persons/zeina-abdallah']

prof_data = []


for row in tqdm(prof_links):
    prof_request = requests.get(row)
    prof_soup = BeautifulSoup(prof_request.content, 'html.parser')
    prof_grid = prof_soup.find('div', { "class" : "profile"})
    
    info = dict()
    prof_info = []
    
    info['url'] = row if row else "-"
    info['image_url'] = "https://pureportal.coventry.ac.uk//"+prof_grid.find('img').attrs['src'] if prof_grid.find('img').attrs['src'] else "-"
    info['name'] = prof_grid.select_one('.person-details h1').text.strip() if prof_grid.select_one('.person-details h1') else "-"
    info['department'] = prof_grid.select_one('ul li a').text.strip() if prof_grid.select_one('ul li a') else "-"
    info['university'] = "Coventry University"
    info['phone'] = prof_grid.select_one('.phones .description').text.strip() if prof_grid.select_one('.phones .description') else "-"
    info['email'] = prof_grid.select_one('.emails .description').text.strip() if prof_grid.select_one('.emails .description') else "-"
    info['fingerprints_url'] = row+"/fingerprints" if row else "-"
    info['projects_url'] = row+"/projects" if row else "-"
    
    prof_info.append(info["url"])
    prof_info.append(info["image_url"])
    prof_info.append(info["name"])
    prof_info.append(info["department"])
    prof_info.append(info["university"])
    prof_info.append(info["phone"])
    prof_info.append(info["email"])
    
    ##########################
    
    prof_info.append(info["fingerprints_url"])
    
    fingerprint_request = requests.get(info["fingerprints_url"])
    fingerprint_soup = BeautifulSoup(fingerprint_request.content, 'html.parser')    
    fingerprint_grid = fingerprint_soup.find('div', { "class" : "person-fingerprints"})
    
    prof_research = []
    
    for research in fingerprint_soup.find_all('span', { "class" : "concept" }):
        prof_research.append(research.text)
    
    prof_info.append(str(prof_research)[1:-1])
    
    ##########################
    
    prof_info.append(info["projects_url"])
    
    project_request = requests.get(info["projects_url"])
    project_soup = BeautifulSoup(project_request.content, 'html.parser')    
    project_grid = project_soup.find('section', { "class" : "list-results"})

    prof_all_project = []
    
    for a in project_grid.find_all('div', { "class" : "result-container" }):

        proj_info = dict()
        
        proj_info['title'] = a.select_one('.title').text.strip() if a.select_one('.title') else "-"
        proj_info['person'] = a.select_one('p span').text.strip() if a.select_one('p span') else "-"
        proj_info['fundings'] = a.select_one('.fundings').text.strip() if a.select_one('.fundings') else "-"
        proj_info['period'] = a.select_one('.period').text.strip() if a.select_one('.period') else "-"
        proj_info['type'] = a.select_one('.type_classification').text.strip() if a.select_one('.type_classification') else "-"

        prof_proj = []
        
        prof_proj.append(proj_info['title'])
        prof_proj.append(proj_info['person'])
        prof_proj.append(proj_info['fundings'])
        prof_proj.append(proj_info['period'])
        prof_proj.append(proj_info['type'])
        
        prof_all_project.append(prof_proj)
    
    prof_info.append(prof_all_project)
    
    ##########################
    
    
    prof_data.append(prof_info)
    

#print(prof_info)
#print(prof_data)

printmd("<span style='font-family:courier;color:green;font-size:115%;' > **--> Profile Details of "+str(len(prof_links))+" Researchers fetched Successfully.**</span>")


Fetching Details of Each Researcher : 


<span style='font-family:courier;color:green;font-size:115%;' > **--> Profile Details of 500 Researchers fetched Successfully.**</span>

In [14]:
prof_df = pd.DataFrame(prof_data, columns= ['profile_url', 'image_url', 'full_name', 'department', 'university', 'contact', 'email', 'fingerprints_url', 'research_topic', 'projects_url', 'project_info'])

prof_df.to_csv (r'C:\\Users\\Saurabh\\Desktop\\IR\\Shruti\\new\\Docs csv etc\\coventry_uni_df.csv', index = False, header=True)

prof_df

,profile_url,image_url,full_name,department,university,contact,email,fingerprints_url,research_topic,projects_url,project_info
0,https://pureportal.coventry.ac.uk/en/persons/m...,https://pureportal.coventry.ac.uk///files-asse...,Mai Abbas,"Centre for Agroecology, Water and Resilience",Coventry University,-,ad1212@coventry.ac.uk,https://pureportal.coventry.ac.uk/en/persons/m...,"'food', 'research interest', 'health', 'sovere...",https://pureportal.coventry.ac.uk/en/persons/m...,[]
1,https://pureportal.coventry.ac.uk/en/persons/m...,https://pureportal.coventry.ac.uk///assets/no-...,Mohammadreza Abbassi Monjezi,Institute for Future Transport and Cities,Coventry University,-,abbassim@coventry.ac.uk,https://pureportal.coventry.ac.uk/en/persons/m...,,https://pureportal.coventry.ac.uk/en/persons/m...,[]
2,https://pureportal.coventry.ac.uk/en/persons/m...,https://pureportal.coventry.ac.uk///files-asse...,Mohsen Abbaszadeh,Faculty Research Centre in Fluid and Complex S...,Coventry University,-,abbaszam@coventry.ac.uk,https://pureportal.coventry.ac.uk/en/persons/m...,"'Solubility', 'Aquifers', 'Wetting', 'Temperat...",https://pureportal.coventry.ac.uk/en/persons/m...,[]
3,https://pureportal.coventry.ac.uk/en/persons/m...,https://pureportal.coventry.ac.uk///files-asse...,Meryem Abdelhafid,"Centre for Trust, Peace and Social Relations",Coventry University,-,abdelha4@coventry.ac.uk,https://pureportal.coventry.ac.uk/en/persons/m...,"'Algerian', 'feminism', 'Religion', 'migration...",https://pureportal.coventry.ac.uk/en/persons/m...,[]
4,https://pureportal.coventry.ac.uk/en/persons/m...,https://pureportal.coventry.ac.uk///assets/no-...,Mohamed Abdelshafy,"School of Computing, Electronics and Maths",Coventry University,-,ad0270@coventry.ac.uk,https://pureportal.coventry.ac.uk/en/persons/m...,"'Routing protocols', 'Network protocols', 'Net...",https://pureportal.coventry.ac.uk/en/persons/m...,[]
...,...,...,...,...,...,...,...,...,...,...,...
495,https://pureportal.coventry.ac.uk/en/persons/s...,https://pureportal.coventry.ac.uk///assets/no-...,Sarah Nnenna Aideyan,School of Marketing and Management,Coventry University,-,ordus@coventry.ac.uk,https://pureportal.coventry.ac.uk/en/persons/s...,,https://pureportal.coventry.ac.uk/en/persons/s...,[]
496,https://pureportal.coventry.ac.uk/en/persons/n...,https://pureportal.coventry.ac.uk///assets/no-...,Nurudeen Aigoro,Institute for Future Transport and Cities,Coventry University,-,aigoron@coventry.ac.uk,https://pureportal.coventry.ac.uk/en/persons/n...,,https://pureportal.coventry.ac.uk/en/persons/n...,[]
497,https://pureportal.coventry.ac.uk/en/persons/t...,https://pureportal.coventry.ac.uk///assets/no-...,Toluwanimi Akaehomen,School of Marketing and Management,Coventry University,-,akaehoment@coventry.ac.uk,https://pureportal.coventry.ac.uk/en/persons/t...,,https://pureportal.coventry.ac.uk/en/persons/t...,[]
498,https://pureportal.coventry.ac.uk/en/persons/e...,https://pureportal.coventry.ac.uk///assets/no-...,Emre Akgun,Institute for Future Transport and Cities,Coventry University,-,akgune@coventry.ac.uk,https://pureportal.coventry.ac.uk/en/persons/e...,,https://pureportal.coventry.ac.uk/en/persons/e...,[]
